In [1]:
#%pip install -qU langchain langchain-ollama langchain-text-splitters langchain_huggingface

### RAG Pipeline with langchain_ollama

In [3]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.embeddings import FakeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
# Step 1: Initialize Ollama Chat Model
# Ensure ollama is running :
#    ollama serve
#    ollama pull mistral:7b
llm = ChatOllama(
    model="mistral:7b",  
    temperature=0.7,
)

In [5]:
# Step 2: Prepare Documents
documents = [
    "Large Language Models (LLMs) are AI systems trained on massive text datasets to understand and generate human language. "
    "They are commonly built using Transformer architectures, which allow them to process context across long sequences of text.",

    "Natural Language Processing (NLP) focuses on enabling computers to interpret and work with human language. "
    "Core NLP tasks include text classification, question answering, sentiment analysis, summarization, and translation.",

    "Python is widely used for building and deploying LLM and NLP systems. "
    "Popular libraries and frameworks include Hugging Face Transformers for model usage, tokenizers for text processing, "
    "and vector databases for retrieval-augmented generation (RAG) workflows."
]

In [6]:
# Step 3: Split Documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
chunks = text_splitter.split_text("\n".join(documents))

In [7]:
# Step 4: Create Vector Store
embeddings = FakeEmbeddings(size=256, model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = InMemoryVectorStore.from_texts(chunks, embeddings)
retriever = vectorstore.as_retriever(k=2)

In [8]:
# Step 5: Define RAG Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system",
        "You are a helpful assistant. Prioritize answering using the provided context. "
        "If the context contains the answer, rely on it strictly. "
        "If the context does not contain the answer, you may use your own verified, "
        "parametric knowledge — but do NOT make up facts. "
        "If you are not confident or the information is unknown, say "
        "'I don't know' or 'The context does not provide this information.'\n\n"
        "Context: {context}"
    ),
    ("human", "{query}")
])


In [9]:
# Step 6: Simple RAG Function
def retrieve_and_respond(query: str) -> str:
    """Retrieve docs → format prompt → invoke LLM"""
    context_docs = retriever.invoke(query)
    context = "\n".join([doc.page_content for doc in context_docs])
    
    messages = prompt.format_messages(context=context, query=query)
    response = llm.invoke(messages)
    return response.content

In [10]:
# Step 7: Multi-turn Conversation
class ConversationPipeline:
    def __init__(self, llm, retriever, prompt):
        self.llm = llm
        self.retriever = retriever
        self.prompt = prompt
        self.chat_history = []
    
    def chat(self, user_query: str) -> str:
        """Handle multi-turn conversation"""
        context_docs = self.retriever.invoke(user_query)
        context = "\n".join([doc.page_content for doc in context_docs])
        
        # Format messages with context and conversation history
        messages = prompt.format_messages(context=context, query=user_query)
        
        response = self.llm.invoke(messages)
        response_text = response.content
        
        # Store in history
        self.chat_history.append({"user": user_query, "assistant": response_text})
        
        return response_text

In [12]:
# Step 8: Run Pipeline
if __name__ == "__main__":
    print("=== Single-turn RAG ===")
    result = retrieve_and_respond("What is a Large Language Model?")
    print(f"Response: {result}\n")
    
    print("=== Multi-turn Conversation ===")
    conversation = ConversationPipeline(llm, retriever, prompt)
    
    queries = [
        "What is a Large Language Model?",
        "What are common tasks in Natural Language Processing?",
        "Can I use Python to build and deploy LLM applications?"
    ]
    
    for query in queries:
        response = conversation.chat(query)
        print(f"User: {query}")
        print(f"Assistant: {response}\n")

=== Single-turn RAG ===
Response:  A Large Language Model (LLM) refers to a type of artificial intelligence model that is designed to process and understand human language. It's capable of tasks such as text classification, question answering, and other natural language processing tasks. The context suggests that Python is often used for building and deploying these models, and they are commonly built using Transformer architectures. However, the context does not provide a specific definition or size limit for what constitutes a "large" language model. For more precise information about large language models, it would be best to consult additional resources or documentation on the topic.

=== Multi-turn Conversation ===
User: What is a Large Language Model?
Assistant:  A Large Language Model (LLM) refers to an AI system that has been trained on vast amounts of text data. These models are designed to understand and generate human-like text based on the input they receive. The context pr

### With Tools (Function Calling)

In [13]:
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

@tool  # Docstring is mandatory as it becomes the tool description given to the LLM
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the result."""
    return a * b

llm_with_tools = llm.bind_tools([multiply])

# Ask model
ai_msg = llm_with_tools.invoke([HumanMessage("What is 5 times 3?")])

# Execute tool
tool_call = ai_msg.tool_calls[0]
result = multiply.invoke(tool_call["args"])

print("Tool result:", result)


Tool result: 15


### Streaming

In [16]:
print("=== Streaming ===")

for chunk in llm.stream("Tell me about Large Language Models"):
    print(chunk.content, end="", flush=True)
print()

=== Streaming ===
 Large Language Models, like me, are a type of artificial intelligence (AI) model designed to process and generate human-like text. These models are trained on vast amounts of internet text data, learning from it to understand, predict, and generate human-like responses to a wide range of inputs.

The training process involves feeding the model with a large dataset of texts, allowing it to learn patterns, grammar rules, and even some facts about the world. However, it's important to note that these models don't have consciousness or emotions, they simply generate text based on patterns they've learned during training.

Large Language Models can be used for various tasks such as answering questions, writing essays, summarizing texts, translating languages, and even creating engaging conversations. They are a significant part of the advancements in AI technology, pushing the boundaries of what machines can do when it comes to natural language understanding and generatio